# Usage
In this tutorial we use [Braess network](https://en.wikipedia.org/wiki/Braess%27s_paradox#Mathematical_approach)
as an example.

## Import Modules
In addition to Graphvar module, Graphvar requires [NetworkX](https://networkx.org/documentation/stable/index.html)
to define the network on that variables are defined. So we should import both.

In [1]:
import os
import sys
import time

sys.path.insert(0, os.path.abspath('../'))

import networkx as nx
import graphvar as gv

## Define Variables
First, create the network with [nx.DiGraph](https://networkx.org/documentation/stable/reference/classes/digraph.html#networkx.DiGraph).


In [2]:
G = nx.DiGraph()
G.add_edges_from([
    ('start', 'A'),
    ('start', 'B') ,
    ('A', 'B'),
    ('A', 'end'),
    ('B', 'end')
])

Note that Graphvar accepts any hashable objects as nodes, as does NetworkX.

Then, create BaseGraph object from the DiGraph instance.

In [3]:
BG = gv.BaseGraph(G)

The BaseGraph object has two roles, which are

* An identifier to identify which network the variable is defined on and
* Memory to store information of the network such as structure, order of edges and nodes, etc.

Finally, create NodeVar instance to define node variables or EdgeVar instance to define edge variables.

In [4]:
od_flow = gv.NodeVar(BG)
print(repr(od_flow))
edge_cost = gv.EdgeVar(BG)
print(repr(edge_cost))

node	value
A	0.0
B	0.0
end	0.0
start	0.0

edge	value
('A', 'B')	0.0
('A', 'end')	0.0
('B', 'end')	0.0
('start', 'A')	0.0
('start', 'B')	0.0



These codes makes variables defined on all nodes or edges of BG,
all of whose values are zero.

You can modify initial values of variables when you create it by giving a
keyword argument ```init_val```.
The argument ```init_val``` accepts several types of variables.
if you want to set all initial values as the same value, simply give a scalar:

In [5]:
od_flow = gv.NodeVar(BG, init_val=10)
print(repr(od_flow))
edge_cost = gv.EdgeVar(BG, init_val=10)
print(repr(edge_cost))

node	value
A	10.0
B	10.0
end	10.0
start	10.0

edge	value
('A', 'B')	10.0
('A', 'end')	10.0
('B', 'end')	10.0
('start', 'A')	10.0
('start', 'B')	10.0



or if you want to set each value in detail, give
* a dictionary that has node- or edge- indexes as keys and initial values as values:

In [6]:
od_flow = gv.NodeVar(BG, init_val={
    'start': -6,
    'A': 0,
    'B': 0,
    'end': 6
})
print(repr(od_flow))
edge_cost = gv.EdgeVar(BG, init_val={
    ('start', 'A'): 0,
    ('start', 'B'): 50 ,
    ('A', 'B'): 10,
    ('A', 'end'): 50,
    ('B', 'end'): 0
})
print(repr(edge_cost))

node	value
A	0.0
B	0.0
end	6.0
start	-6.0

edge	value
('A', 'B')	10.0
('A', 'end')	50.0
('B', 'end')	0.0
('start', 'A')	0.0
('start', 'B')	50.0



* a nx.DiGraph that have same network structure as BG
and have 'value' attribute on all nodes or edges:

In [7]:
IniG = nx.DiGraph(BG)
IniG.add_nodes_from([
    ('start', {'value': -6}),
    ('A', {'value': 0}),
    ('B', {'value': 0}),
    ('end', {'value': 6})
])
IniG.add_edges_from([
    ('start', 'A', {'value': 0}),
    ('start', 'B', {'value': 50}),
    ('A', 'B', {'value': 10}),
    ('A', 'end', {'value': 50}),
    ('B', 'end', {'value': 0})
])
od_flow = gv.NodeVar(BG, init_val=IniG)
print(repr(od_flow))
edge_cost = gv.EdgeVar(BG, init_val=IniG)
print(repr(edge_cost))

node	value
A	0.0
B	0.0
end	6.0
start	-6.0

edge	value
('A', 'B')	10.0
('A', 'end')	50.0
('B', 'end')	0.0
('start', 'A')	0.0
('start', 'B')	50.0



* a NodeVar or EdgeVar object (initializing by them is faster than that
by dictionary or nx.DiGraph.)

In [8]:
new_od_flow = gv.NodeVar(BG, init_val=od_flow)
print(repr(new_od_flow))
new_edge_cost = gv.EdgeVar(BG, init_val=edge_cost)
print(repr(new_edge_cost))

node	value
A	0.0
B	0.0
end	6.0
start	-6.0

edge	value
('A', 'B')	10.0
('A', 'end')	50.0
('B', 'end')	0.0
('start', 'A')	0.0
('start', 'B')	50.0



Note that you can also modify values after creating instances by ```set_value()``` method.

In [9]:
new_od_flow.set_value('A', 100)
print(repr(new_od_flow))
new_edge_cost.set_value(('A', 'B'), 100)
print(repr(new_edge_cost))


node	value
A	100.0
B	0.0
end	6.0
start	-6.0

edge	value
('A', 'B')	100.0
('A', 'end')	50.0
('B', 'end')	0.0
('start', 'A')	0.0
('start', 'B')	50.0



## Mathematical Operations

NodeVar and EdgeVar objects can be added to, subtracted from, multiplied by and divided
by another objects of the same classes.

In [10]:
print(repr(new_od_flow + od_flow))
print(repr(new_od_flow - od_flow))
print(repr(new_od_flow * od_flow))
print(repr(new_od_flow / od_flow))  # this raises warnings because of the zero division

node	value
A	100.0
B	0.0
end	12.0
start	-12.0

node	value
A	100.0
B	0.0
end	0.0
start	0.0

node	value
A	0.0
B	0.0
end	36.0
start	36.0

node	value
A	inf
B	nan
end	1.0
start	1.0



/Users/watanabe/Documents/GitHub/graphvar/graphvar/classes.py:172: RuntimeWarning: divide by zero encountered in true_divide
  res_array = operation_func(other._array)
/Users/watanabe/Documents/GitHub/graphvar/graphvar/classes.py:172: RuntimeWarning: invalid value encountered in true_divide
  res_array = operation_func(other._array)


NodeVar and EdgeVar objects also operated with scalar values.

In [11]:
print(repr(new_od_flow + 5))
print(repr(new_od_flow - 5))
print(repr(new_od_flow * 5))
print(repr(new_od_flow / 5))

node	value
A	105.0
B	5.0
end	11.0
start	-1.0

node	value
A	95.0
B	-5.0
end	1.0
start	-11.0

node	value
A	500.0
B	0.0
end	30.0
start	-30.0

node	value
A	20.0
B	0.0
end	1.2
start	-1.2



## Computational Efficiency

NodeVar and EdgeVar stores variables' values as np.ndarray and
the mathematical operations shown above are operated with these arrays.

In [12]:
print(new_od_flow.array)  # You can see the array by .array property.
new_od_flow.array[1] = 5  # .array is read-only
print(new_od_flow.array)

[100.   0.   6.  -6.]
[100.   0.   6.  -6.]


Thus, these operation is as fast as that of np.ndarray.
The larger the network is, the smaller the difference between the speed of
these two methods are.

In [13]:
# Create a huge graph to show computational efficiency.
import random
import numpy as np
import time
import timeit

G = nx.DiGraph()
G.add_nodes_from(list(range(10000)))
for i in range(20000):
    edge = random.sample(G.nodes, 2)
    G.add_edge(*edge)
bg = gv.BaseGraph(G)
timeit_args = {
    'timer': time.process_time, 'number': 100000, 'globals': globals()
}

In [14]:
print("calculation with NodeVar ============")
e1 = gv.NodeVar(bg, init_val = 1)
e2 = gv.NodeVar(bg, init_val = 2.5739)
print(timeit.timeit("e1 + e2", **timeit_args))
print("calculation with np.ndarray =========")
a1 = e1.array
a2 = e2.array
print(timeit.timeit("a1 + a2", **timeit_args))

calculation with NodeVar ============
0.5916939999999999
calculation with np.ndarray =========
0.3572320000000002


In [15]:
print("calculation with EdgeVar ============")
e1 = gv.EdgeVar(bg, init_val = 1)
e2 = gv.EdgeVar(bg, init_val = 2.5739)
print(timeit.timeit("e1 + e2", **timeit_args))
print("calculation with np.ndarray =========")
a1 = e1.array
a2 = e2.array
print(timeit.timeit("a1 + a2", **timeit_args))

calculation with EdgeVar ============
0.8706019999999999
calculation with np.ndarray =========
0.6312820000000006


In [ ]:
print("calculation with graphvar ============")
e = gv.EdgeVar(bg, init_val = 1)
A = gv.IncidenceMatrix(bg)
print(timeit.timeit("A @ e", **timeit_args))
print("calculation with np.ndarray =========")
e = e.array
A = A.matrix
print(timeit.timeit("A @ e", **timeit_args))

Specifically, these examples show that the Graphvar operations are
approx 2.0e-7 cpu-seconds slower than np.ndarray operations, regardless of
the size of the network and the types of the operation.